# Performance & Risk Data

In MongoDB, all data of a particular `fund` at a particular `date` is stored in a single document. Values are stored as an Embedded Document and can be nested further.

```javascript
{
    _id: ObjectId('...')
    date: 2023-12-31T00:00:00.000+00:00
    fundId: ObjectId('...')
    values: {
        nav: 100
        region: {
            AMER: 0.3
            APAC: 0.2
            EMEA: 0.1
        }
    }
}
```

Import the libraries and connect to the database

In [117]:
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson import ObjectId
client = MongoClient(os.environ["MONGODB_HOST"])
db = client.ftkdb

## Saving Data

In [118]:
# index is Timestamp, row is Series
# values can be an empty dict
def save_data(fund, data):
    for date, row in data.iterrows():    
        row.index = row.index.map(lambda x: f'values.{x}')
        db.data.update_one({
            'fundId': fund['_id'],
            'date': date
        },{
            '$set': {**row.dropna().to_dict()}
        },
        upsert=True)

In [119]:
dates = pd.date_range('2023-12-31', periods=3, freq='M')
navs = pd.DataFrame(np.random.randint(100, 110, size=3), index=dates, columns=['nav'])

fund1 = db.fund.find_one({'name': 'Fund One'})
fund2 = db.fund.find_one({'name': 'Fund Two'})
fund3 = db.fund.find_one({'name': 'Fund Three'})
fund4 = db.fund.find_one({'name': 'Fund Four'})

save_data(fund1, navs)
save_data(fund3, navs)

In [120]:
# Period cannot be encoded
rets = navs.pct_change().to_period()
rets = rets.to_timestamp()
rets.index = rets.index + pd.offsets.MonthEnd(0)
rets.columns = ['return']

save_data(fund2, rets)
save_data(fund3, rets)

In [121]:
regions = pd.DataFrame(np.random.rand(3, 3), index=dates, columns=['region.AMER', 'region.EMEA', 'region.APAC'])
save_data(fund1, regions)

In [122]:
multi = pd.concat([navs, rets, regions], axis=1)
save_data(fund4, multi)

## Loading Data

1. Single fund

In [123]:
data = list(db.data.find({'fundId': fund1['_id']}).sort({'date': 1}))
df = pd.DataFrame([d['values'] for d in data], index=[d['date'] for d in data])

navs = df.loc[:, 'nav']
regions = df.loc[:, 'region'].apply(pd.Series)

2. Load data of all funds including fund name

In [124]:
from datetime import datetime, timezone
pipeline = [
    {
        '$match': {
            'date': {
                '$gte': datetime(2023, 12, 31, 0, 0, 0, tzinfo=timezone.utc)
            }
        }
    }, {
        '$lookup': {
            'from': 'fund', 
            'localField': 'fundId', 
            'foreignField': '_id', 
            'as': 'fund'
        }
    }, {
        '$unwind': {
            'path': '$fund'
        }
    }, {
        '$project': {
            '_id': 0, 
            'date': 1, 
            'fundId': 1, 
            'fundName': '$fund.name', 
            'values': 1
        }
    }
]

raw = pd.json_normalize(db.data.aggregate(pipeline))

In [125]:
# List all risk data type, but lots of NaN
sparse = raw.set_index(['fundId','fundName','date'])
sparse.to_excel('sparse.xlsx')

In [126]:
dense = raw.melt(id_vars=['date','fundId','fundName']).dropna().pivot(index=['fundId','fundName','variable'], columns='date')
dense.columns = dense.columns.droplevel(0)
dense.to_excel('dense.xlsx')

In [127]:
def to_sparse(data):
    """Preprocess for updating database"""
    data.columns.name = 'date'
    data = data.T.reset_index().melt(id_vars=['date']).pivot(index=['fundId','fundName','date'], columns='variable')
    data.columns = data.columns.droplevel(0)
    return data

def to_dense(data):
    data = data.reset_index().melt(id_vars=['date','fundId','fundName']).dropna().pivot(index=['fundId','fundName','variable'], columns='date')
    data.columns = data.columns.droplevel(0)
    return data

def import_data(path):
    data = pd.read_excel(path, index_col=[0,1,2])
    if data.index.names == ['fundId', 'fundName', 'variable']:
        # Convert dense format to sparse format
        data = to_sparse(data)
    if data.index.names == ['fundId', 'fundName', 'date']:
        for index, row in data.iterrows():
            fund_id = index[0]
            date = index[2]
            db.data.update_one({
                'fundId': ObjectId(fund_id),
                'date': date
            },{
                '$set': {**row.dropna().to_dict()}
            },
            upsert=True)            

df1_dense = pd.read_excel('dense.xlsx', index_col=[0,1,2]).sort_index()
df1_sparse = pd.read_excel('sparse.xlsx', index_col=[0,1,2]).sort_index()

df2_dense = to_dense(df1_sparse).sort_index()
df2_sparse = to_sparse(df1_dense).sort_index()

pd.testing.assert_frame_equal(df1_dense, df2_dense)
pd.testing.assert_frame_equal(df1_sparse, df2_sparse, check_names=False)

import_data('dense.xlsx')
import_data('sparse.xlsx')